#Computationally-driven Identification of Antibody Epitopes

Authors: Casey K Hua, Albert T Gacerez, Charles L Sentman, Margaret E Ackerman, Yoonjoo Choi, Chris Bailey-Kellogg -  
Dartmouth College, United States; Korea Advanced Institute of Science and Technology (KAIST), Republic of Korea.
https://doi.org/10.7554/eLife.29023.001

Understanding where antibodies recognize antigens can help define mechanisms of action and provide insights into progression of immune responses. The authors investigated the extent to which information about binding specificity implicitly encoded in amino acid sequence can be leveraged to identify antibody epitopes.

In computationally-driven epitope localization, possible antibody–antigen binding modes are modeled, and targeted panels of antigen variants are designed to experimentally test these hypotheses. Prospective application of this approach to two antibodies enabled epitope localization using five or fewer variants per antibody, or alternatively, a six-variant panel for both simultaneously.

Retrospective analysis of a variety of antibodies and antigens demonstrated an almost 90% success rate with an average of three antigen variants, further supporting the observation that the combination of computational modeling and protein design can reveal key determinants of antibody–antigen binding and enable efficient studies of collections of antibodies identified from polyclonal samples or engineered libraries.
https://elifesciences.org/articles/29023

#Overview of computationally-driven epitope identification by EpiScope.

(A) Ab–Ag docking models are generated using computational docking methods. In the example, the green structure is the Ag human IL-18 (PDB ID: 2VXT:A), while the cartoons represent possible poses of the Ab (limited here to three for clarity). Full details including docking models and designs for this example are provided in a PyMol session. (B) Ag variants containing a pre-defined numbers of mutations (here triple mutations, colored triangles) are generated for each docking model. (C) Variants are clustered with respect to spatial locations in the Ag, and a set of variants predicted to disrupt all of the docking models is selected. (D) Ag mutagenesis and Ag-Ab binding experiments are performed to identify which mutations result in loss of Ab recognition. (E) Examination of the disruptive variant(s) enables localization of the Ab epitope in terms of both mutated positions (pink balls) and consistent docking models, here with the model (light pink cartoon) quite similar to the actual crystal structure (dark pink cartoon).

https://doi.org/10.7554/eLife.29023.002

![](https://iiif.elifesciences.org/lax:29023%2Felife-29023-fig1-v2.tif/full/617,/0/default.webp)https://elifesciences.org/articles/29023

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns #visualization
import matplotlib.pyplot as plt #visualization
%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py
import plotly.express as px

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#The Three files: Covid, B Cell and Sars.

In [ ]:
df = pd.read_csv('../input/epitope-prediction/input_sars.csv', encoding='ISO-8859-2')
df.head()

In [ ]:
df1 = pd.read_csv('../input/epitope-prediction/input_covid.csv', encoding='ISO-8859-2')
df1.head()

In [ ]:
df2 = pd.read_csv('../input/epitope-prediction/input_bcell.csv', encoding='ISO-8859-2')
df2.head()

#Spoiler alert: No missing Values.

In [ ]:
#Code by Akash Sahoo https://www.kaggle.com/akashdotcom/covid-19-sars-b-cell-predictions-using-nn-eda/notebook

#training data created
frames = [df2, df]
bcell_sars_df = pd.concat(frames, axis=0, ignore_index=True)
bcell_sars_df.head() 

#Antibody Epitope Prediction - SOURCE http://tools.iedb.org/bcell/help/

I. Methods for predicting continuous antibody epitope from protein sequences

General basis: Parameters such as hydrophilicity, flexibility, accessibility, turns, exposed surface, polarity and antigenic propensity of polypeptides chains have been correlated with the location of continuous epitopes. This has led to a search for empirical rules that would allow the position of continuous epitopes to be predicted from certain features of the protein sequence.

General method: When computing the score for a given residue i, the amino acids in an interval of the chosen length, centered around residue i, are considered. In other words, for a window size n, the i - (n-1)/2 neighboring residues on each side of residue i were used to compute the score for residue i. Unless specified, the score for residue i is the average of the scale values for these amino acids (see table 1 for specific method implementation details). In general, a window size of 5 to 7 is appropriate for finding regions that may potentially be antigenic.

Methods that were included in the files: Chou and Fasman ;; Emini;; Kolaskar and Tongaonkar;; Parker.

Methods that were not in this Dataset files: Karplus and Schulz ;; Bepipred-1.0 Linear Epitope Prediction;; BepiPred-2.0: Sequential B-Cell Epitope Predictor.

In [ ]:
#checking for null values
bcell_sars_df.isna().sum()

In [ ]:
#Code by Akash Sahoo https://www.kaggle.com/akashdotcom/covid-19-sars-b-cell-predictions-using-nn-eda/notebook
#shuffling the dataset
bcell_sars_df = bcell_sars_df.sample(frac=1).reset_index(drop = True)

In [ ]:
#checking the target variable countplot
sns.set_style('darkgrid')
sns.countplot(bcell_sars_df['target'],linewidth=3,palette="Set2",edgecolor='black')

In [ ]:
# Distribution of different type of amount
fig , ax = plt.subplots(1,3,figsize = (12,5))

start_position = df.start_position.values
end_position = df.end_position.values
target = df.target.values

sns.distplot(start_position , ax = ax[0] , color = 'blue').set_title('B Cell Start Position' , fontsize = 14)
sns.distplot(end_position , ax = ax[1] , color = 'cyan').set_title('B Cell End Position' , fontsize = 14)
sns.distplot(target , ax = ax[2] , color = 'purple').set_title('B Cell Target' , fontsize = 14)


plt.show()

In [ ]:
#Code by Akash Sahoo https://www.kaggle.com/akashdotcom/covid-19-sars-b-cell-predictions-using-nn-eda/notebook

idx_train = bcell_sars_df['target'].astype("bool").values
fig, axes = plt.subplots(2, 3,figsize=(16,8))
sns.set_style('darkgrid')
axes = [x for a in axes for x in a]
for i,name in enumerate(["kolaskar_tongaonkar", "chou_fasman", "hydrophobicity", "stability", "parker", "emini"]):
    value = bcell_sars_df[name]
    sns.distplot(value[~idx_train],ax = axes[i], color='red')
    sns.distplot(value[idx_train],ax = axes[i], color = 'blue')
    axes[i].set_xlabel(name,fontsize=12)
    fig.legend(labels = ["target 0","target 1"],loc="right",fontsize=12)

#Chou and Fasman beta turn prediction

Reference: Chou PY, Fasman GD. Prediction of the secondary structure of proteins from their amino acid sequence. Adv Enzymol Relat Areas Mol Biol. 1978;47:45-148.

Description: The rationale for predicting turns to predict antibody epitopes is based on the paper by Pellequer et al, Immunology Letters. Instead of implementing the turn scale of that paper which has some non-standard properties, the authors decided to use the Chou and Fasman scale which is commonly used to predict beta turns .http://tools.iedb.org/bcell/help/

Secondary structural prediction of proteins from their amino acid sequence, by Peter Y. Chou and Gerald D. Fasman

Predicted secondary structures of proteins (α-helix, β-pleated sheet and β-turns) give insight into the understanding of protein folding and biological activity.https://www.sciencedirect.com/science/article/abs/pii/0968000477904406

In [ ]:
# Plot of Chou Fasman Method
plt.style.use("classic")
sns.distplot(df['chou_fasman'], color='blue')
plt.title(f"Chou Fasman [\u03BC : {df['chou_fasman'].mean():.2f} status | \u03C3 : {df['chou_fasman'].std():.2f} status]")
plt.xlabel("Chou Fasman Method")
plt.ylabel("Count")
plt.show()

#Emini surface accessibility scale

Reference: Emini EA, Hughes JV, Perlow DS, Boger J. Induction of hepatitis A virus-neutralizing antibody by a virus-specific synthetic peptide. J Virol. 1985 Sep;55(3):836-9.

Description: The calculation was based on surface accessibility scale on a product instead of an addition within the window. The accessibility profile was obtained using the formulae Sn = (n+4+i ) (0.37)-6 where Sn is the surface probability, dn is the fractional surface probability value, and i vary from 1 to 6.

A hexapeptide sequence with Sn greater than 1.0 indicates an increased probability for being found on the surface.http://tools.iedb.org/bcell/help/

In [ ]:
# Plot of Emini Method
plt.style.use("classic")
sns.distplot(df['emini'], color='red')
plt.title(f"Emini [\u03BC : {df['emini'].mean():.2f} status | \u03C3 : {df['emini'].std():.2f} status]")
plt.xlabel("Emini Method")
plt.ylabel("Count")
plt.show()

#Kolaskar and Tongaonkar antigenicity scale

Reference: Kolaskar AS, Tongaonkar PC. A semi-empirical method for prediction of antigenic determinants on protein antigens. FEBS Lett. 1990 Dec 10;276(1-2):172-4.

Description: A semi-empirical method which makes use of physicochemical properties of amino acid residues and their frequencies of occurrence in experimentally known segmental epitopes was developed to predict antigenic determinants on proteins. 

Application of this method to a large number of proteins has shown by the authors that the method can predict antigenic determinants with about 75% accuracy which is better than most of the known methods.http://tools.iedb.org/bcell/help/

In [ ]:
# Plot of Kolaskar Tongaonkar Method
plt.style.use("classic")
sns.distplot(df['kolaskar_tongaonkar'], color='green')
plt.title(f"kolaskar_tongaonkar [\u03BC : {df['kolaskar_tongaonkar'].mean():.2f} status | \u03C3 : {df['kolaskar_tongaonkar'].std():.2f} status]")
plt.xlabel("Kolaskar Tongaonkar Method")
plt.ylabel("Count")
plt.show()

#Parker Hydrophilicity Prediction

Reference: Parker JM, Guo D, Hodges RS. New hydrophilicity scale derived from high-performance liquid chromatography peptide retention data: correlation of predicted surface residues with antigenicity and X-ray-derived accessible sites. Biochemistry. 1986 Sep 23; 25(19):5425-32.

Description: In this method, hydrophilic scale based on peptide retention times during high-performance liquid chromatography (HPLC) on a reversed-phase column was constructed.

A window of seven residues was used for analyzing epitope region. The corresponding value of the scale was introduced for each of the seven residues and the arithmetical mean of the seven residue value was assigned to the fourth, (i+3), residue in the segment.http://tools.iedb.org/bcell/help/

Since hydrophilicity parameters have been used extensively in algorithms to predict which amino acids residues are antigenic, the authors have compared the profiles generated by their new set of hydrophilic HPLC (high-performance liquid chromatography) parameters on the same scale as nine other sets of parameters.

In [ ]:
# Plot of Parker Method
plt.style.use("classic")
sns.distplot(df['parker'], color='purple')
plt.title(f"Parker [\u03BC : {df['parker'].mean():.2f} status | \u03C3 : {df['parker'].std():.2f} status]")
plt.xlabel("Parker Method")
plt.ylabel("Count")
plt.show()

In [ ]:
##https://www.kaggle.com/ilyabiro/visual-analysis-or-how-to-find-out-who-escaped-1st/notebook
fig, ax = plt.subplots(figsize=(10, 8))
ax.set_title('Methods distribution')
g = sns.kdeplot(df['chou_fasman'].loc[df['emini'] == 1], 
                shade= True, ax=ax, label='emini').set_xlabel('chou_fasman')
g = sns.kdeplot(df['chou_fasman'].loc[df['emini'] == 0], 
                shade=True, ax=ax, label='emini')
ax.grid()

In [ ]:
#https://www.kaggle.com/ilyabiro/visual-analysis-or-how-to-find-out-who-escaped-1st/notebook
fig, axarr = plt.subplots(1, 2, figsize=(14,8))
axarr[0].set_title('Kolaskar Tongaonkar Method Distribution')
f = sns.distplot(df['kolaskar_tongaonkar'], color='g', bins=15, ax=axarr[0])
axarr[1].set_title('Methods ')
g = sns.kdeplot(df['kolaskar_tongaonkar'].loc[df['target'] == 1], 
                shade= True, ax=axarr[1], label='kolaskar_tongaonkar').set_xlabel('Parker & Kolaskar Tongaonkar Methods')
g = sns.kdeplot(df['parker'].loc[df['target'] == 0], 
                shade=True, ax=axarr[1], label='parker')

#The snippet above was made for Titanic Competition, therefore the KDE was suppose to be used when you have sort of Yes/No, Survived/Not Survived. Otherwise, the Kde will sink. Just like mine above.  

In [ ]:
import matplotlib.style as style

train_heat=df[df["target"].notnull()]
train_heat=train_heat.drop(["target"],axis=1)
style.use('ggplot')
sns.set_style('whitegrid')
plt.subplots(figsize = (20,8))
## Plotting heatmap. 

# Generate a mask for the upper triangle (taken from seaborn example gallery)
mask = np.zeros_like(train_heat.corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True


sns.heatmap(train_heat.corr(), 
            cmap=sns.diverging_palette(150, 275, l=60, n=7), 
            mask = mask, 
            annot=True, 
            center = 0, 
           );
## Give title. 
plt.title("Heatmap of all the Features", fontsize = 30);

#Isoelectric Point Estimation, Amino Acid Sequence and Algorithms, By Amanda Maxwell

The isoelectric point, or pI,represents a point of balance for a molecule, where the external surface charge is a net zero. This factor governs electrophoretic mobility in proteins and also plays a role in identifying peptides from mass spectral proteomics data.

pI depends on a number of factors, including amino acid sequence, post-translational modifications (PTMs) and presence of side chain—all of which can alter surface charge and behavior depending on the pH of the environment.

Various methods for predicting pI in denatured proteins exist, and most base this calculation on amino acid sequence with reference to pKa values recorded for ionizable constituents. Although these predictive methods exist, their performance can be variable and may skew ensuing results.https://www.thermofisher.com/blog/proteomics/isoelectric-point-estimation-amino-acid-sequence-and-algorithms/

In [ ]:
fig = px.bar(df, 
             x='chou_fasman', y='isoelectric_point', color_discrete_sequence=['crimson'],
             title='Chou and Fasman Scale & Isoelectric Point', text='peptide_seq')

fig.show()

#Hydrophobic strip-of-helix algorithm for selection of T cell-presented peptides

Authors: C J Stille , L J Thomas, V E Reyes, R E Humphreys - PMID: 2825000 DOI: 10.1016/0161-5890(87)90068-x

In extension of the hypothesis that an amphipathic alpha helix of Ii (Phe146-Val164) bound to the foreign antigen-presenting site (desetope) of class II MHC molecules through hydrophobic amino acid residues (Phe146, Leu150, Leu153, Met157, Ile160, Val164) which were present in an axial strip along one side of the Ii helix, the authors developed an algorithm to search for T cell-presented peptides showing a similar hydrophobic strip-of-helix. 

Such peptides might bind to the class II MHC molecule site which was complementary to the Ii hydrophobic strip-of-helix. The strip-of-helix hydrophobicity index was the mean hydrophobicity (from Kyte-Doolittle values) of sets of amino acids in axial strips down sides of helices for 3-6 turns, at positions, n, n + 4, N + 7, n + 11, n + 14, and n + 18.

Peptides correlating well with T cell responsiveness had: (1) 12-19 amino acids (3-5 cycles or 4-6 turns of an alpha helix), (2) a strip with highly hydrophobic residues, (3) adjacent, moderately hydrophilic strips, and (4) no prolines.

The degree of hydrophilicity of the remainder of a putative antigenic helix above a threshold value did not count in this index. That is, the magnitude of amphipathicity was not judged to be the principal selecting factor for T cell-presented peptides.

This simple algorithm to quantitate strip-of-helix hydrophobicity in a putative amphipathic alpha helix, allowing otherwise generally hydrophilic residues, predicted 10 of 12 T cell-presented peptides in seven well-studied proteins. The derivation and application of this algorithm were analyzed.https://pubmed.ncbi.nlm.nih.gov/2825000/

In [ ]:
fig = px.bar(df, 
             x='hydrophobicity', y='parker', color_discrete_sequence=['darkgreen'],
             title='Parker Method Hydrophilic Scale', text='peptide_seq')
fig.show()

#The PentaFOLD 3.0 Algorithm for the selection of Stable Elements of Secondary Structure to be included in Vaccine Peptides

Author(s): Vladislav Victorovich Khrustalev - Journal Name: Protein & Peptide Letters -
DOI : 10.2174/0929866527666201110123851

Aims: The aim of this study was to create a new version of the PentaFOLD algorithm and to test its performance experimentally in several proteins and peptides.

Background: Synthetic vaccines can cause production of neutralizing antibodies only in case if short peptides form the same secondary structure as fragments of full-length proteins. The PentaFOLD 3.0 algorithm was designed to check stability of alpha helices, beta strands, and random coils using several propensity scales obtained during analysis of 1730 3D structures of proteins.

Objective: The algorithm has been tested in the three peptides known to keep the secondary structure of the corresponding fragments of full-length proteins: the NY25 peptide from the Influenza H1N1 hemagglutinin, the SF23 peptide from the diphtheria toxin, the NQ21 peptide from the HIV1 gp120; as well as in the CC36 peptide from the human major prion protein.

Method: Affine chromatography for antibodies against peptides accompanied by circular dichroism and fluorescence spectroscopy were used to check the predictions of the algorithm.

Result: Immunological experiments showed that all abovementioned peptides are more or less immunogenic in rabbits. The fact that antibodies against the NY25, the SF23, and the NQ21 form stable complexes with corresponding full-length proteins has been confirmed by affine chromatography. The surface of SARS CoV-2 spike receptor-binding domain interacting with hACE2 has been shown to be unstable according to the results of the PentaFOLD 3.0.

Conclusion: The PentaFOLD 3.0 algorithm (http://chemres.bsmu.by/PentaFOLD30.htm) can be used with the aim to design vaccine peptides with stable secondary structure elements.
https://www.eurekaselect.com/node/187782/article/the-pentafold-30-algorithm-for-the-selection-of-stable-elements-of-secondary-structure-to-be-included-in-vaccine-peptides

In [ ]:
fig = px.bar(df, 
             x='stability', y='emini', color_discrete_sequence=['purple'],
             title='Emini Method & Stability', text='peptide_seq')
fig.show()

#Epitope Predictions Indicate the Presence of Two Distinct Types of Epitope-Antibody-Reactivities Determined by Epitope Profiling of Intravenous Immunoglobulins

Authors: Mitja Luštrek,  ,  Peter Lorenz,  Michael Kreutzer,  Zilliang Qian,  ,  Felix Steinbeck,  Di Wu,  ,  Nadine Born,  Bjoern Ziems,  Michael Hecker,  Miri Blank,  Yehuda Shoenfeld,  Zhiwei Cao,  Michael O. Glocker,  Yixue Li,  Georg Fuellen,  and Hans-Jürgen Thiesen.

PLoS One. 2013; 8(11): e78605.
Published online 2013 Nov 11. doi: 10.1371/journal.pone.0078605

Epitope-antibody-reactivities (EAR) of intravenous immunoglobulins (IVIGs) determined for 75,534 peptides by microarray analysis demonstrate that roughly 9% of peptides derived from 870 different human protein sequences react with antibodies present in IVIG.

Computational prediction of linear B cell epitopes was conducted using machine learning with an ensemble of classifiers in combination with position weight matrix (PWM) analysis. Machine learning slightly outperformed PWM with area under the curve (AUC) of 0.884 vs. 0.849.
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3823795/

![](https://journals.plos.org/plosone/article/figure/image?size=inline&id=info:doi/10.1371/journal.pone.0078605.t004)https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0078605

#The unexpectedly high classification cross-validation accuracy on the unclassifiable peptides.

The 1st degree classifiable and unclassifiable peptides are exactly the opposite of each other with respect to the attributes most useful for classification. For example, if a peptide is classifiable, it is likely to bind if it has a HIGH AROMACITY; if it is unclassifiable, it is likely to bind if it has a LOW Aromaticity.

Because of this opposite behavior, a classifier that correctly classifies one of the two groups must fail on the other. The classifier that was trained on the whole training set correctly classified the larger group (classifiable) and failed on the smaller one (unclassifiable). However, when the classifier that was trained on the 1st degree unclassifiable peptides was used, it no longer faced the contradiction between the groups and thus performed well.
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3823795/

In [ ]:
fig = px.bar(df, 
             x='kolaskar_tongaonkar', y='aromaticity', color_discrete_sequence=['#2B3A67'],
             title='Kolaskar and Tongaonkar Antigenicity Scale', text='end_position')
fig.show()

#B cell epitopes

B cell epitopes are most commonly discontinuous (also called conformational or assembled), consisting of segments of multiple chains brought together by the folding of the protein (antigen). Only about 10% of all epitopes recognized by antibodies are thought to be continuous (also called linear or sequential).

Current state-of-the-art epitope prediction generally uses machine learning approaches. Larsen et al. trained a HIDDEN MARKOV MODEL (HMM) on epitopes in conjunction with random amino acid sequences. The antigenicity of amino acids was then derived from the ratios of their emission probabilities by the epitope HMM and the random HMM. The area under curve (AUC) of receiver operating characteristic (ROC) of such a classifier in combination with a hydrophobicity scale was 0.671. 

Söllner and Mayer used DECISION TREE and NEAREST NEIGHBOR MACHINE LEARNING algorithms. They utilized over 1,000 attributes related to relative positions of amino acids in the sequences and over 250 propensity scales. The post test probability (a measure similar to accuracy) of the best classifier was 69.31%. In the studies cited below, the classifiers used have been trained and tested on B cell epitopes from the Bcipep database as positive examples and random peptides as negative examples (except for Rubinstein et al., who apparently used epitope and non-epitope parts of the same antigens as positive and negative examples). 

Saha and Raghava  employed a RECURRENT NEURAL NETWORK. The accuracy of the trained network was 65.93%.

Chen et al. used the SUPPORT VECTOR MACHINE LEARNING algorithm. Their attributes were frequencies of amino acid pairs and five propensity scores. The resulting accuracy was 73.71%.

El-Manzalawy et al., used SUPPORT VECTOR machines in combination with a subsequence kernel. Their attributes were all subsequences of peptides up to a certain length, including subsequences with gaps. The AUC of the subsequence kernel classification was 0.812 and the accuracy 73.37% (original data set). They reimplemented Chen et al.’s method, which yielded an AUC of 0.717 and an accuracy of 65.65%. The difference compared to the accuracy reported by Chen et al. is probably due to differences in the exact composition of the data set. Results reported by El-Manzalawy et al. confirm the superiority of the subsequence kernel classification.

Rubinstein et al. used the NAIVE BAYES MACHINE LEARNING algorithm. Their attributes were the frequencies of amino acids, the structural properties of proteins and a number of propensity scores. With all these attributes, the AUC was 0.55 and the accuracy was 70.6%. If only the best attributes were selected, the AUC and the accuracy increased to 0.59 and 80.4%, respectively. This large increase (in accuracy) may be due to the fact that apparently the whole data set was used in attribute selection instead of only the training set.https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3823795/

In [ ]:
fig = px.histogram(df1, x="parker", color="hydrophobicity", title='Parker Method Hydrophilic Scale')
fig.update_layout(hovermode='x')
fig.show()

In [ ]:
fig = px.violin(df1, y="kolaskar_tongaonkar", color="isoelectric_point", title='Kolaskar and Tongaonkar Antigenicity Scal')
fig.show()

In [ ]:
#Code by Olga Belitskaya https://www.kaggle.com/olgabelitskaya/sequential-data/comments
from IPython.display import display,HTML
c1,c2,f1,f2,fs1,fs2=\
'#eb3434','#eb3446','Akronim','Smokum',30,15
def dhtml(string,fontcolor=c1,font=f1,fontsize=fs1):
    display(HTML("""<style>
    @import 'https://fonts.googleapis.com/css?family="""\
    +font+"""&effect=3d-float';</style>
    <h1 class='font-effect-3d-float' style='font-family:"""+\
    font+"""; color:"""+fontcolor+"""; font-size:"""+\
    str(fontsize)+"""px;'>%s</h1>"""%string))
    
    
dhtml('Marília Prata, @mpwolke Was here' )